# SceneXplain: Alt-tagger

In this notebook we will use Jina AI's [SceneXplain](https://scenex.jina.ai) to cycle through a zip file of images and generate an alt-tag for each one. The output will be stored in a CSV file then re-zipped with the images.

We will assume that the images are in a zip file. We will:

- Unzip the images
- Send them to SceneXplain to get alt-tags
- Write the alt-tags to a CSV file, along with the image filenames
- Re-zip the file

## SceneXplain JSON Store

We will use SceneXplain's JSON Store to pull a predefined schema that is specialized for generating alt-tags. This is just a simple use case for the JSON Store, so be sure to browse and explore what else you can do with it!

Our JSON Schema looks something like this:

```json
{
  "type": "object",
  "properties": {
    "alt_tag": {
      "type": "string",
      "description": "the most concise description possible of the image’s purpose. If the image is purely decorative (e.g. part of the website's design, not content), leave empty. Do not include text like 'this image contains' or 'image depicts'"
    }
  }
}
```

Below we've defined two JSON Schema IDs, one for general alt-tagging, and one specifically targeted for alt-tagging screenshots.

In [114]:
# json_schema = "43IxSufD5tpWYCPkMOj8"  # generic alt-tagger
json_schema = "WJ84ezfnag0zWxgkmDJY"  # screenshot alt-tagger

## User options

In [115]:
# filenames
zip_input = "images.zip"
zip_output = "images_with_alt_tags.zip"
csv_filename = "alt_tags.csv"

## Basic setup

Functions to send generate image lists, populate the data to send to SceneXplain, send that data.

In [87]:
import os
import json
import base64
import http.client
from pprint import pprint
from glob import glob
import csv
import tempfile

In [7]:
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("Environment variables loaded from .env")
except ImportError:
    os.environ['SCENEX_SECRET'] = "<your SceneXplain key>"

Environment variables loaded from .env


In [98]:
def extract_zip(zip_file):
    temp_dir = tempfile.mkdtemp()
    with zipfile.ZipFile(zip_input, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

        return temp_dir

In [29]:
def generate_image_list(folder_name, max_count=100):
    filetypes = ['jpg', 'jpeg', 'png']
    image_files = []
    
    for filetype in filetypes:
        image_files.extend(glob(f'{folder_name}/*.{filetype}'))

    return image_files[:max_count]

In [113]:
# SceneX setup and functions

SCENEX_SECRET=os.getenv('SCENEX_SECRET')
features = ["json"]

scenex_headers = {
    "x-api-key": f"token {SCENEX_SECRET}",
    "content-type": "application/json",
}

def image_to_data_uri(file_path):
    with open(file_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode("utf-8")
        return f"data:image/png;base64,{encoded_image}"
        
def generate_scenex_data(image_files, json_schema=None, question=None, features=[]):
    data = {}
    data['data'] = []

    for file in image_files:
        cid = file.split('/')[-1]
        row = {
            "image": image_to_data_uri(file),
            "features": ["json"],
            "cid": cid,
            "json_schema_id": json_schema
        }

        if question:
            row["question"] = question

        if json_schema:
            row["json_schema"] = json_schema

        data['data'].append(row)

    return data

def process_scenex(data):
    connection = http.client.HTTPSConnection("api.scenex.jina.ai")
    connection.request("POST", "/v1/describe", json.dumps(data), scenex_headers)
    response = connection.getresponse()
    response_data = response.read().decode("utf-8")
    
    connection.close()

    return json.loads(response_data)['result']

## Creating our alt-tags

### Unzip images

Unzip the images into a temporary directory which we'll clean up later.

In [109]:
import zipfile

In [110]:
temp_dir = extract_zip(zip_input)

### Generate data

Let's take the image list and add our SceneXplain options (e.g. JSON Schema ID, image data URI, etc).

In [111]:
image_files = generate_image_list(temp_dir)

In [112]:
scenex_data = generate_scenex_data(image_files, json_schema=json.dumps(schema), features=features)

### Send data to SceneXplain

In [14]:
scenex_response = process_scenex(scenex_data)

### Write to CSV

In [34]:
headers = ["filename", "alt-tag"]

In [104]:
def generate_csv(csv_filename, image_files, scenex_response):
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for filename, row in zip(image_files, scenex_response):
            # print(row)
            alt_tag = json.loads(row['i18n']['en'])['alt_tag']
            row = [filename, alt_tag]
            writer.writerow(row)

In [72]:
generate_csv(csv_filename, image_files, scenex_response)

### Re-zip the files

In [107]:
with zipfile.ZipFile(zip_output, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_filename)

    for foldername, subfolders, filenames in os.walk(folder_name):
        for filename in filenames:
            file_path = os.path.join(foldername, filename)
            zipf.write(file_path, os.path.relpath(file_path, folder_name))

### Clean up

In [103]:
import shutil
shutil.rmtree(temp_dir)